In [ ]:
%use dataframe, kandy

# How are we doing?
We wouldn't be working agile if we didn't have some ways to gather timely feedback that drive us towards improvements.

For this we'll use "Aging Charts".

In [ ]:
import kotlinx.datetime.daysUntil
import org.jetbrains.kotlinx.dataframe.api.dropNulls

val csv = DataFrame.read("data/a_team.csv")

In [ ]:
val cleaned = csv.dropNulls { `In Analysis` and Analyzed and `In Development` and Developed and `In Acceptance` and `In Production` }
cleaned

# Current state of our KanBan board
We have 3 Work items In Progress:

ST-0179 has been in progress for 36 days and is currently In Acceptance.

ST-0180 has been in progress for 4 days and is currently In Development.

ST-0181 has been in progress for 4 days and is currently In Analysis.


# Can we improve anything?

How can we know?

In [ ]:
val offsetWhenFinishedSameDay = 1

val valueStream = cleaned
    .add("AnalysisTime") { `In Analysis`.date.daysUntil(Analyzed.date) + offsetWhenFinishedSameDay }
    .add("CycleTime") { `In Analysis`.date.daysUntil(`In Production`.date) + offsetWhenFinishedSameDay }
    .select("StoryId", "AnalysisTime","CycleTime")
valueStream

In [ ]:
val percentiles = valueStream.aggregate {
    percentile(50.0) { AnalysisTime } into "p50Analysis"
    percentile(85.0) { AnalysisTime } into "p85Analysis"
    percentile(50.0) { CycleTime } into "p50CycleTime"
    percentile(85.0) { CycleTime } into "p85CycleTime"
}
percentiles

In [ ]:
import org.jetbrains.kotlinx.kandy.ir.Plot
import org.jetbrains.kotlinx.statistics.distribution.NormalDistribution
import org.jetbrains.letsPlot.Stat
import org.jetbrains.letsPlot.core.spec.plotson.BinStatOptions
import org.jetbrains.letsPlot.core.spec.plotson.theme

val analysisAgingChart = plot(valueStream) {
    layout { size = 2000 to 900 }
    points {
        y(AnalysisTime)
        x(StoryId)
    }
    hLine { yIntercept.constant(percentiles["p50Analysis"]); type = LineType.DASHED }
    hLine { yIntercept.constant(percentiles["p85Analysis"]); type = LineType.DASHED }
}
analysisAgingChart

In [ ]:
val agingChart = plot(valueStream) {
    layout { size = 2000 to 900 }
    points {
        y(CycleTime)
        x(StoryId)
    }
    // vertical markers
    hLine { yIntercept.constant(percentiles["p50CycleTime"]); type = LineType.DASHED }
    hLine { yIntercept.constant(percentiles["p85CycleTime"]); type = LineType.DASHED }
}
agingChart

# How good busy are we?
Here's our current state again.

## ST-0179 has been in progress for **36 days** and is currently In Acceptance.

According to our Aging Chart our 50th percentile is 24 days "in progress", and the 85th percentile is 33.
We sure don't want this story to mess up our stats!
So let's get it to production asap!
This needs to be our first priority.

## ST-0180 has been in progress for 4 days and is currently In Development.
This story is currently well below the 50th percentile of 24 days, and it's right on the median of our analysis time, so probably no action required today.

## ST-0181 has been in progress for 4 days and is currently In Analysis.
Given that our median is 4 days and 85th is 6 days, we should look into finishing analysis for this one today.
If your gut says we won't be done today, let's put effort into splitting up this story today instead.